In [1]:
import cv2
import numpy as np

def stitch(image1, image2):
    # Feature detection and description
    sift = cv2.SIFT_create()
    keypoints1, descriptors1 = sift.detectAndCompute(image1, None)
    keypoints2, descriptors2 = sift.detectAndCompute(image2, None)

    # Draw keypoints on images (for visualization)
    image1_with_keypoints = cv2.drawKeypoints(image1, keypoints1, None)
    image2_with_keypoints = cv2.drawKeypoints(image2, keypoints2, None)
    cv2.imwrite('image1_with_keypoints.jpg', image1_with_keypoints)
    cv2.imwrite('image2_with_keypoints.jpg', image2_with_keypoints)
    
    # BFMatcher with default params
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)
    
    # Apply ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)
    
    # RANSAC-based homography estimation
    src_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    H, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    
    # Warp image1 onto image2
    height, width = image2.shape[:2]
    width += image1.shape[1]
    warped_image = cv2.warpPerspective(image2, H, (width, height))
    
    warped_image[:,:image1.shape[1]]=image1
    
    return warped_image

# Load images
image_paths = ['VS_Batch2_1.jpg', 'VS_Batch2_2.jpg', 'VS_Batch2_3.jpg']
images = [cv2.imread(path) for path in image_paths]

# Start stitching from the last photo
n = len(images)
final_panorama = images[n-1]

for i in range( n-2 , -1 , -1):
    final_panorama = stitch(images[i],final_panorama)

# Display the final stitched panorama
cv2.imwrite('Final_Panorama.jpg', final_panorama)


True